In [65]:
import torch
import json
import transformer as t
import importlib
importlib.reload(t);

In [75]:
# read in file
with open('commits.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(list(set(text)))

# Build encoder and decoder
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

# create data tensors
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
training_data = data[:n]
validation_data = data[n:]

In [76]:
#
# Hyper parameter
#
batch_size      = 64
max_iterations  = 10000
eval_interval   = 3000
learning_rate   = 1e-4
eval_iters      = 200

In [83]:
#
# Network 
#
t.vocabulary_size           = len(chars)
t.attention_heads_per_block = 16
t.attention_blocks          = 16
t.sample_size               = 64     # number of consecutive characters to predict from
t.embedding_size            = 128    # size of the embedding vectors
t.dropout                   = 0.2

torch.manual_seed(1337)

model = t.Transformer()
m = model.to(t.device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
sum(p.nelement() for p in model.parameters())

3202155

In [84]:
#
# mini-batch creation
#
def get_batch(split):
    data = training_data if split == 'train' else validation_data
    ix = torch.randint(len(data) - t.sample_size, (batch_size, ))
    x = torch.stack([data[i:i+t.sample_size] for i in ix])
    y = torch.stack([data[i+1:i+t.sample_size+1] for i in ix])
    x,y = x.to(t.device), y.to(t.device)
    return x,y
#
# evaluation
#
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            x, y = get_batch(split)
            _, loss = model(x, y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [82]:
#
# Training
#
model.train()
for iter in range(max_iterations):
    if iter % eval_interval == 0 and iter > 0:
        print("evaluating...")
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    if iter % 200 == 0:
        print(f"step {iter}")

    xb, yb = get_batch('train')
    _, loss = model(xb,yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

step 0


KeyboardInterrupt: 

In [72]:
model.eval()
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long, device=t.device), max_tokens=500)[0].tolist()))


|Oncertvitgedeloafin lerege ndeang rag
| Dteiprps AleryDi Iret tconiteealiGs iesthrgenktomen Rens inChapanTenzun; rt opes Bueswas Imag
| DLPrtür
|||renerWImgteceliopomeppisurlfiomande Less andesaungok-- sideCoSliepaktenkonged Tes SeicktterErvang r
||Unesstromtstiprns T™
|Mahites-EMamaneitttfs]$/CouninPrWiu_uNTasQungeniesepuin Dtondackeigen lepreses u-Trbed q den Majen Papises
|g SerDPruchttieryCochtrtdert
{205.08.200/tfin;Cs/tfs:/om- UNzti48088.
| Pteasseri Listorunta
ncervon-TeTlerich
{02.09.20


In [63]:
model.eval()
dummy_input = torch.randint(low=0, high=t.vocabulary_size, size=(1, t.sample_size), dtype=torch.long)
torch.onnx.export(model, dummy_input, "commits.onnx")

============= Diagnostic Run torch.onnx.export version 2.0.1+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [70]:
dictionary = {str(k): v for k, v in itos.items()}
with open('vocabulary.json', 'w') as file:
    json.dump(dictionary, file)